In [46]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from scipy import stats
df = pd.read_csv('heart_failure_clinical_records_dataset.csv')
df.head(5)

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [47]:
df.info(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   age                       299 non-null    float64
 1   anaemia                   299 non-null    int64  
 2   creatinine_phosphokinase  299 non-null    int64  
 3   diabetes                  299 non-null    int64  
 4   ejection_fraction         299 non-null    int64  
 5   high_blood_pressure       299 non-null    int64  
 6   platelets                 299 non-null    float64
 7   serum_creatinine          299 non-null    float64
 8   serum_sodium              299 non-null    int64  
 9   sex                       299 non-null    int64  
 10  smoking                   299 non-null    int64  
 11  time                      299 non-null    int64  
 12  DEATH_EVENT               299 non-null    int64  
dtypes: float64(3), int64(10)
memory usage: 30.5 KB


In [48]:
## Seleccionamos X e Y en nuestro conjunto (Y es la columna que representa la clase)
x = df.drop('DEATH_EVENT', axis=1)
y = df['DEATH_EVENT']

## Y separamos el conjunto en entrenamiento y testing
## Deberíamos probar variando test_size y random_state
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.2, random_state=4);

In [102]:
# Creamos una clase llamada KNN que va a almacenar todos los procedimientos necesarios para realizar el cálculo
class kNN():
    # Colocamos por defecto el valor de K = 3, p = None (para distancia de Minkowsky) y le decimos que la métrica de distancia por defecto es la euclídea
    def __init__(self, metrica='euclidea', k=3, p=None):
        self.metrica = metrica
        self.k=k
        self.p=p

    # con esta función vamos a cargar los conjuntos de entrenamiento
    def cargarDatos(self, X_train, Y_train):
        self.X_train = X_train
        self.Y_train = Y_train

    def distancia(self, metrica, v1, v2, p=2):
        return {
            'euclidea': lambda: np.sqrt(np.sum((v1-v2)**2)),
            'minkowsky': lambda: np.sum(np.abs(v1-v2)**p)**(1/p),
            'manhattan': lambda: np.sum(np.abs(v1-v2)),
        }.get(metrica, lambda: None)
    
    # def distanciaEuclidea(self, v1, v2):
    #     return np.sqrt(np.sum((v1-v2)**2))

    # def distanciaMinkowski(self, v1, v2, p=2):
    #     return np.sum(np.abs(v1-v2)**p)**(1/p)
    
    # def distanciaManhattan(self, v1, v2):
    #     return np.sum(np.abs(v1-v2))

    # La función calcularVecinos toma un set d y obtiene sus k vecinos cercanos
    def calcularVecinos(self, d):
        # inicializamos un vector de distancias vacío
        distancias = list()

        # iremos calculando fila por fila en nuestro conjunto de entrenamiento
        for (fila, clase) in zip(self.X_train, self.Y_train):
            # dependiendo de la métrica de distancia que el usuario eligió
            dist = self.distancia(self.metrica, fila, d, self.p)()
            distancias.append((dist, clase))

        # Ordenamos la lista por el valor de la distancia
        distancias.sort(key=lambda x: x[0])

        # Obtenemos los K vecinos cercanos
        vecinos = list()
        for i in range(self.k):
            vecinos.append(distancias[i][1])

        return vecinos

    # Necesitamos una función que se encargue de predecir, nos va a servir para obtener la exactitud que tiene cada métrica en este conjunto
    def predecir(self, X_test):
        preds = []
        predicciones = []
        # Calculamos los vecinos para cada fila del conjunto de entrenamiento
        for fila in X_test:
            vecinos = self.calcularVecinos(fila)
            moda = stats.mode(vecinos)[0]
            predicciones.append(moda)
        return np.array(predicciones)

In [110]:
def calcularExactitud(predicciones, Y_test):
    return 100 * (predicciones == Y_test).mean()

pred = [];

# Calculamos predicciones para distancia euclídea y de manhattan con k de 3, 4 y 5
for metrica in ['euclidea', 'manhattan']:
    for k in [3, 4, 5]:
        nn = kNN(k=k, metrica=metrica);
        nn.cargarDatos(X_train.values, Y_train.values);
        pred.append(
            dict(
                {
                    'metrica': metrica,
                    'k': k,
                    'p': None,
                    'exactitud': calcularExactitud(nn.predecir(X_test.values), Y_test)
                }
            )
        )

# Calculamos predicciones para distancia de Minkowsky con k de 3, 4 y 5, y con un p de 2, 3 y 4
for k in [3, 4, 5]:
    for p in [2, 3, 4]:
        nn = kNN(k=k, metrica='minkowsky', p=p);
        nn.cargarDatos(X_train.values, Y_train.values);
        pred.append(
            dict(
                {
                    'metrica': 'minkowsky',
                    'k': k,
                    'p': p,
                    'exactitud': calcularExactitud(nn.predecir(X_test.values), Y_test)
                }
            )
        )

# Ahora podemos ver los diferentes valores
for prediccion in pred:
    print(f"Metrica: {prediccion['metrica']}. K: {prediccion['k']}. P: {prediccion['p']}. Exactitud: {prediccion['exactitud']}.")

Metrica: euclidea. K: 3. P: None. Exactitud: 55.00000000000001.
Metrica: euclidea. K: 4. P: None. Exactitud: 66.66666666666666.
Metrica: euclidea. K: 5. P: None. Exactitud: 63.33333333333333.
Metrica: manhattan. K: 3. P: None. Exactitud: 55.00000000000001.
Metrica: manhattan. K: 4. P: None. Exactitud: 66.66666666666666.
Metrica: manhattan. K: 5. P: None. Exactitud: 61.66666666666667.
Metrica: minkowsky. K: 3. P: 2. Exactitud: 55.00000000000001.
Metrica: minkowsky. K: 3. P: 3. Exactitud: 55.00000000000001.
Metrica: minkowsky. K: 3. P: 4. Exactitud: 55.00000000000001.
Metrica: minkowsky. K: 4. P: 2. Exactitud: 66.66666666666666.
Metrica: minkowsky. K: 4. P: 3. Exactitud: 65.0.
Metrica: minkowsky. K: 4. P: 4. Exactitud: 65.0.
Metrica: minkowsky. K: 5. P: 2. Exactitud: 63.33333333333333.
Metrica: minkowsky. K: 5. P: 3. Exactitud: 63.33333333333333.
Metrica: minkowsky. K: 5. P: 4. Exactitud: 61.66666666666667.
